In [6]:
import sys, os, multiprocessing
import boto3

import pandas as pd
#import geopandas as gpd
#import numpy as np

import h3

AWS_S3_BUCKET = 'wbg-geography01'
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_SESSION_TOKEN = os.getenv("AWS_SESSION_TOKEN")

In [29]:
# Search through AWS folder and see update date for completed zonal layers
bucket = "wbg-geography01"
prefix = "Space2Stats/h3_stats_data/GLOBAL/Urbanization_Pop/"
region = "us-east-1"
s3client = boto3.client("s3", region_name=region, verify=False)

# Loop through the S3 bucket and get all the file keys
more_results = True
try:
    del token  # noqa
except Exception:
    pass
loops = 0

all_res = []
while more_results:
    print(f"Completed loop: {loops}")
    if loops > 0:
        objects = s3client.list_objects_v2(
            Bucket=bucket,
            ContinuationToken=token,  # noqa
            Prefix=prefix,  # noqa
        )
    else:
        objects = s3client.list_objects_v2(Bucket=bucket, Prefix=prefix)
    more_results = objects["IsTruncated"]
    if more_results:
        token = objects["NextContinuationToken"]
    loops += 1
    for res in objects["Contents"]:
        all_res.append(res)

Completed loop: 0


c:\wbg\Anaconda3\envs\s2s\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wbg-geography01.s3.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [30]:
s3_df = pd.DataFrame(all_res)
s3_df.sort_values("LastModified", inplace=True)
s3_df['Day_Processed'] = s3_df['LastModified'].apply(lambda x: str(x.month) + "_" + str(x.day))
s3_df['Day_Processed'].value_counts()


Day_Processed
11_14    270
11_16    251
11_15    229
11_13     92
Name: count, dtype: int64

In [ ]:
ntl_files[0]

'https://globalnightlight.s3.amazonaws.com/composites/j01_202306_ops/DNB_j01_20230601-20230630_global_ecm-slcorr_v10_ops.avg_rade9.tif'